<a href="https://colab.research.google.com/github/BrianKani/GIS-CropForecast/blob/master/data3(clip).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import geemap
import ee

In [2]:
ee.Authenticate()
ee.Initialize(project="ee-briankipsang19")

In [3]:
import time
import pandas as pd
import numpy as np



In [9]:
def export_oneimage(img, folder, name, scale, crs):
    task = ee.batch.Export.image(img, name, {
        'driveFolder': folder,
        'driveFileNamePrefix': name,
        'scale': scale,
        'crs': crs
    })
    task.start()
    while task.status()['state'] == 'RUNNING':
        print('Running...')
        # Perhaps task.cancel() at some point.
        time.sleep(10)
    print('Done.', task.status())

# Read the locations CSV file
locations = pd.read_csv('/content/locations.csv', header=None)

# Transforms an Image Collection with 1 band per Image into a single Image with items as bands
def appendBand(current, previous):
    # Rename the band
    previous = ee.Image(previous)
    current = current.select([0,1,2,3,4,5,6])
    # Append it to the result (Note: only return current item on first element/iteration)
    accum = ee.Algorithms.If(ee.Algorithms.IsEqual(previous, None), current, previous.addBands(ee.Image(current)))
    # Return the accumulation
    return accum

# Define the bounding box coordinates for Uasin Gishu County
uasin_gishu_bbox = [[35.3, 0.6], [35.4, 0.6], [35.4, 0.7], [35.3, 0.7]]

# Filter the MODIS image collection to include only images within Uasin Gishu County and within a specific date range
imgcoll = ee.ImageCollection('MODIS/MOD09A1') \
    .filterBounds(ee.Geometry.Polygon(uasin_gishu_bbox)) \
    .filterDate('2016-12-31', '2022-08-04')

img = imgcoll.iterate(appendBand)
img = ee.Image(img)

img_0 = ee.Image(ee.Number(-100))
img_16000 = ee.Image(ee.Number(16000))

img = img.min(img_16000)
img = img.max(img_0)

# Iterate over locations and export images
for row in locations.values:
    loc1, loc2, lat, lon = row[0], row[1], row[2], row[3]
    fname = '{}_{}'.format(int(loc1), int(loc2))

    scale = 1000
    crs = 'EPSG:4326'

    # Define the region of interest as the bounding box of Uasin Gishu County
    region = str(uasin_gishu_bbox)

    while True:
        try:
            export_oneimage(img, 'test', fname, scale, crs)
        except Exception as e:
            print('Error:', e)
            print('Retry...')
            time.sleep(10)
            continue
        break


Done. {'state': 'READY', 'description': '1_1', 'creation_timestamp_ms': 1707460910728, 'update_timestamp_ms': 1707460910728, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': '6YKQKDBXOB2B6KVXFVZBOVDD', 'name': 'projects/ee-briankipsang19/operations/6YKQKDBXOB2B6KVXFVZBOVDD'}
Done. {'state': 'READY', 'description': '1_2', 'creation_timestamp_ms': 1707460911228, 'update_timestamp_ms': 1707460911228, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'QZNE4JQNYUZQWDWKO2BW2HK4', 'name': 'projects/ee-briankipsang19/operations/QZNE4JQNYUZQWDWKO2BW2HK4'}
Done. {'state': 'READY', 'description': '1_3', 'creation_timestamp_ms': 1707460911760, 'update_timestamp_ms': 1707460911760, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'SB7OCPQNCUO7SORYTKH42DSW', 'name': 'projects/ee-briankipsang19/operations/SB7OCPQNCUO7SORYTKH42DSW'}
Done. {'state': 'READY', 'description': '1_4', 'creation_timestamp_ms': 1707460912349, 'update_timestamp_ms': 1707460912349, 'start_timesta